# Kaggle LANL Earthquake Prediction Modeling

### Code Setup

Below are the imports needed to run the code.  The code has been written and run in Python 3.6 and 3.7 Anaconda environments.  Many of these libraries request a citation when used in an academic paper.  Note the use of the Scikit-Learn (Pedregosa et al. (2011), XGBoost (Chen & Guestrin, 2016) and LightGBM (Ke, et al., 2017) libraries for machine learning and support.  Numpy is utilized to provide many numerical functions for feature creation (van der Walt, Colbert & Varoquaux, 2011). Pandas is very helpful for its ability to support data manipulation and feature creation (McKinney, 2010).  SciPy is utilized to provide signal processing functions, especially filtering and for Pearson's correlation metrics (Jones E., et al, 2001).  The Jupyter environment in which this project is presented is a descendant of the IPython environment originated by Pérez & Granger (2007).

In [77]:
import os
import time
import feather
import warnings
import traceback
import numpy as np
import pandas as pd
from scipy.stats import pearsonr
from scipy import stats
import scipy.signal as sg
import multiprocessing as mp
from scipy.signal import hann
from scipy.signal import hilbert
from scipy.signal import convolve
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

import xgboost as xgb
import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

from tqdm import tqdm, tqdm_notebook
warnings.filterwarnings("ignore")

Define some constants.
The signal constants define how the signal and Fourier transforms will be filtered to produce bandwidth limited features.

In [2]:
OUTPUT_DIR = '../final_model/'  # set for local environment
DATA_DIR = '../input/'  # set for local environment

SIG_LEN = 150000
NUM_SEG_PER_PROC = 6000
NUM_THREADS = 6

NY_FREQ_IDX = 75000  # the test signals are 150k samples long, Nyquist is thus 75k.
CUTOFF = 18000
MAX_FREQ_IDX = 20000
FREQ_STEP = 2500

### Feature Creation

Function to split the raw data into 6 groups for later multiprocessing.  The feature builder function took so long that it was run as 6 concurrent processes in order to speed it up. This perhaps could have been more easily acomplished with the "skiprows" and "nrows" parameters of the Python Pandas read csv function rather than creating 6 new files. 

In [ ]:
def split_raw_data():

    df = feather.read_dataframe(os.path.join(DATA_DIR, 'train.feather'))
    #df = pd.read_csv(os.path.join(DATA_DIR, 'train.csv'))

    max_start_index = len(df.index) - SIG_LEN
    slice_len = int(max_start_index / NUM_THREADS)

    for i in tqdm_notebook(range(NUM_THREADS)):
        df0 = df.iloc[slice_len * i: (slice_len * (i + 1)) + SIG_LEN]
        #df0.to_csv(os.path.join(DATA_DIR, 'raw_data_%d.csv' % i), index=False)
        feather.write_dataframe(df0, os.path.join(DATA_DIR, f'raw_data_{i}.feather'))

In [ ]:
split_raw_data()

Build six sets of random indices.  Stratified random sampling will be performed on the data.  This is for several reasons.  It ensures relatively even coverage of the width of the input signal and it allows for multiprocessing so that the script runs in a reasonable time.  Also, working on data chunks that represent only a portion of the very large input data set means that the whole data set is not loaded into memory multiple times (once for each process).  This makes the feature building more memory efficient.  All of this helps to avoid crashes and allow the feature building portion of the script to run overnight.

In [ ]:
def build_rnd_idxs():
    rnd_idxs = np.zeros(shape=(NUM_THREADS, NUM_SEG_PER_PROC), dtype=np.int32)
    max_start_idx = 100000000

    for i in tqdm_notebook(range(NUM_THREADS)):
        np.random.seed(5591 + i)
        start_indices = np.random.randint(0, max_start_idx, size=NUM_SEG_PER_PROC, dtype=np.int32)
        rnd_idxs[i, :] = start_indices

    np.savetxt(fname=os.path.join(OUTPUT_DIR,'start_indices_4k.csv'),
               X=np.transpose(rnd_idxs),
               fmt='%d', delimiter=',')

In [ ]:
build_rnd_idxs()

Helper functions for feature generation.  These were sourced from a Kaggle kernel script (Preda, 2019).  The "sta_lta" refers to the short term average divided by the long term average.  The trend feature is a linear regression on a portion of the signal.

In [3]:
def add_trend_feature(arr, abs_values=False):
    idx = np.array(range(len(arr)))
    if abs_values:
        arr = np.abs(arr)
    lr = LinearRegression()
    lr.fit(idx.reshape(-1, 1), arr)
    return lr.coef_[0]

def classic_sta_lta(x, length_sta, length_lta):
    sta = np.cumsum(x ** 2)
    # Convert to float
    sta = np.require(sta, dtype=np.float)
    # Copy for LTA
    lta = sta.copy()
    # Compute the STA and the LTA
    sta[length_sta:] = sta[length_sta:] - sta[:-length_sta]
    sta /= length_sta
    lta[length_lta:] = lta[length_lta:] - lta[:-length_lta]
    lta /= length_lta
    # Pad zeros
    sta[:length_lta - 1] = 0
    # Avoid division by zero by setting zero values to tiny float
    dtiny = np.finfo(0.0).tiny
    idx = lta < dtiny
    lta[idx] = dtiny
    return sta / lta

def des_bw_filter_lp(cutoff=CUTOFF):  # low pass filter
    b, a = sg.butter(4, Wn=cutoff/NY_FREQ_IDX)
    return b, a

def des_bw_filter_hp(cutoff=CUTOFF):  # high pass filter
    b, a = sg.butter(4, Wn=cutoff/NY_FREQ_IDX, btype='highpass')
    return b, a

def des_bw_filter_bp(low, high):  # band pass filter
    b, a = sg.butter(4, Wn=(low/NY_FREQ_IDX, high/NY_FREQ_IDX), btype='bandpass')
    return b, a

In [4]:
def create_features_pk_det(seg_id, seg, X, st, end):
    try:
        X.loc[seg_id, 'seg_id'] = np.int32(seg_id)
        X.loc[seg_id, 'seg_start'] = np.int32(st)
        X.loc[seg_id, 'seg_end'] = np.int32(end)
    except:
        pass
    
    sig = pd.Series(seg['acoustic_data'].values)
    b, a = des_bw_filter_lp(cutoff=18000)
    sig = sg.lfilter(b, a, sig)

    peakind = []
    noise_pct = .001
    count = 0

    while len(peakind) < 12 and count < 24:
        peakind = sg.find_peaks_cwt(sig, np.arange(1, 16), noise_perc=noise_pct, min_snr=4.0)
        noise_pct *= 2.0
        count += 1

    if len(peakind) < 12:
        print('Warning: Failed to find 12 peaks for {seg_id}')

    while len(peakind) < 12:
        peakind.append(149999)

    df_pk = pd.DataFrame(data={'pk': sig[peakind], 'idx': peakind}, columns=['pk', 'idx'])
    df_pk.sort_values(by='pk', ascending=False, inplace=True)

    for i in range(0, 12):
        X.loc[seg_id, f'pk_idx_{i}'] = df_pk['idx'].iloc[i]
        X.loc[seg_id, f'pk_val_{i}'] = df_pk['pk'].iloc[i]

    return X

In [5]:
def create_features(seg_id, seg, X, st, end):
    try:
        X.loc[seg_id, 'seg_id'] = np.int32(seg_id)
        X.loc[seg_id, 'seg_start'] = np.int32(st)
        X.loc[seg_id, 'seg_end'] = np.int32(end)
    except:
        pass

    xc = pd.Series(seg['acoustic_data'].values)
    xcdm = xc - np.mean(xc)

    b, a = des_bw_filter_lp(cutoff=18000)
    xcz = sg.lfilter(b, a, xcdm)

    zc = np.fft.fft(xcz)
    zc = zc[:MAX_FREQ_IDX]

    # FFT transform values
    realFFT = np.real(zc)
    imagFFT = np.imag(zc)

    freq_bands = [x for x in range(0, MAX_FREQ_IDX, FREQ_STEP)]
    magFFT = np.abs(zc)
    phzFFT = np.angle(zc)
    phzFFT[phzFFT == -np.inf] = -np.pi / 2.0
    phzFFT[phzFFT == np.inf] = np.pi / 2.0
    phzFFT = np.nan_to_num(phzFFT)

    for freq in freq_bands:
        X.loc[seg_id, 'FFT_Mag_01q%d' % freq] = np.quantile(magFFT[freq: freq + FREQ_STEP], 0.01)
        X.loc[seg_id, 'FFT_Mag_10q%d' % freq] = np.quantile(magFFT[freq: freq + FREQ_STEP], 0.1)
        X.loc[seg_id, 'FFT_Mag_90q%d' % freq] = np.quantile(magFFT[freq: freq + FREQ_STEP], 0.9)
        X.loc[seg_id, 'FFT_Mag_99q%d' % freq] = np.quantile(magFFT[freq: freq + FREQ_STEP], 0.99)
        X.loc[seg_id, 'FFT_Mag_mean%d' % freq] = np.mean(magFFT[freq: freq + FREQ_STEP])
        X.loc[seg_id, 'FFT_Mag_std%d' % freq] = np.std(magFFT[freq: freq + FREQ_STEP])
        X.loc[seg_id, 'FFT_Mag_max%d' % freq] = np.max(magFFT[freq: freq + FREQ_STEP])

        X.loc[seg_id, 'FFT_Phz_mean%d' % freq] = np.mean(phzFFT[freq: freq + FREQ_STEP])
        X.loc[seg_id, 'FFT_Phz_std%d' % freq] = np.std(phzFFT[freq: freq + FREQ_STEP])

    X.loc[seg_id, 'FFT_Rmean'] = realFFT.mean()
    X.loc[seg_id, 'FFT_Rstd'] = realFFT.std()
    X.loc[seg_id, 'FFT_Rmax'] = realFFT.max()
    X.loc[seg_id, 'FFT_Rmin'] = realFFT.min()
    X.loc[seg_id, 'FFT_Imean'] = imagFFT.mean()
    X.loc[seg_id, 'FFT_Istd'] = imagFFT.std()
    X.loc[seg_id, 'FFT_Imax'] = imagFFT.max()
    X.loc[seg_id, 'FFT_Imin'] = imagFFT.min()

    X.loc[seg_id, 'FFT_Rmean_first_6000'] = realFFT[:6000].mean()
    X.loc[seg_id, 'FFT_Rstd__first_6000'] = realFFT[:6000].std()
    X.loc[seg_id, 'FFT_Rmax_first_6000'] = realFFT[:6000].max()
    X.loc[seg_id, 'FFT_Rmin_first_6000'] = realFFT[:6000].min()
    X.loc[seg_id, 'FFT_Rmean_first_18000'] = realFFT[:18000].mean()
    X.loc[seg_id, 'FFT_Rstd_first_18000'] = realFFT[:18000].std()
    X.loc[seg_id, 'FFT_Rmax_first_18000'] = realFFT[:18000].max()
    X.loc[seg_id, 'FFT_Rmin_first_18000'] = realFFT[:18000].min()

    del xcz
    del zc

    b, a = des_bw_filter_lp(cutoff=2500)
    xc0 = sg.lfilter(b, a, xcdm)

    b, a = des_bw_filter_bp(low=2500, high=5000)
    xc1 = sg.lfilter(b, a, xcdm)

    b, a = des_bw_filter_bp(low=5000, high=7500)
    xc2 = sg.lfilter(b, a, xcdm)

    b, a = des_bw_filter_bp(low=7500, high=10000)
    xc3 = sg.lfilter(b, a, xcdm)

    b, a = des_bw_filter_bp(low=10000, high=12500)
    xc4 = sg.lfilter(b, a, xcdm)

    b, a = des_bw_filter_bp(low=12500, high=15000)
    xc5 = sg.lfilter(b, a, xcdm)

    b, a = des_bw_filter_bp(low=15000, high=17500)
    xc6 = sg.lfilter(b, a, xcdm)

    b, a = des_bw_filter_bp(low=17500, high=20000)
    xc7 = sg.lfilter(b, a, xcdm)

    b, a = des_bw_filter_hp(cutoff=20000)
    xc8 = sg.lfilter(b, a, xcdm)

    sigs = [xc, pd.Series(xc0), pd.Series(xc1), pd.Series(xc2), pd.Series(xc3),
            pd.Series(xc4), pd.Series(xc5), pd.Series(xc6), pd.Series(xc7), pd.Series(xc8)]

    for i, sig in enumerate(sigs):
        X.loc[seg_id, 'mean_%d' % i] = sig.mean()
        X.loc[seg_id, 'std_%d' % i] = sig.std()
        X.loc[seg_id, 'max_%d' % i] = sig.max()
        X.loc[seg_id, 'min_%d' % i] = sig.min()

        X.loc[seg_id, 'mean_change_abs_%d' % i] = np.mean(np.diff(sig))
        X.loc[seg_id, 'mean_change_rate_%d' % i] = np.mean(np.nonzero((np.diff(sig) / sig[:-1]))[0])
        X.loc[seg_id, 'abs_max_%d' % i] = np.abs(sig).max()
        X.loc[seg_id, 'abs_min_%d' % i] = np.abs(sig).min()

        X.loc[seg_id, 'std_first_50000_%d' % i] = sig[:50000].std()
        X.loc[seg_id, 'std_last_50000_%d' % i] = sig[-50000:].std()
        X.loc[seg_id, 'std_first_10000_%d' % i] = sig[:10000].std()
        X.loc[seg_id, 'std_last_10000_%d' % i] = sig[-10000:].std()

        X.loc[seg_id, 'avg_first_50000_%d' % i] = sig[:50000].mean()
        X.loc[seg_id, 'avg_last_50000_%d' % i] = sig[-50000:].mean()
        X.loc[seg_id, 'avg_first_10000_%d' % i] = sig[:10000].mean()
        X.loc[seg_id, 'avg_last_10000_%d' % i] = sig[-10000:].mean()

        X.loc[seg_id, 'min_first_50000_%d' % i] = sig[:50000].min()
        X.loc[seg_id, 'min_last_50000_%d' % i] = sig[-50000:].min()
        X.loc[seg_id, 'min_first_10000_%d' % i] = sig[:10000].min()
        X.loc[seg_id, 'min_last_10000_%d' % i] = sig[-10000:].min()

        X.loc[seg_id, 'max_first_50000_%d' % i] = sig[:50000].max()
        X.loc[seg_id, 'max_last_50000_%d' % i] = sig[-50000:].max()
        X.loc[seg_id, 'max_first_10000_%d' % i] = sig[:10000].max()
        X.loc[seg_id, 'max_last_10000_%d' % i] = sig[-10000:].max()

        X.loc[seg_id, 'max_to_min_%d' % i] = sig.max() / np.abs(sig.min())
        X.loc[seg_id, 'max_to_min_diff_%d' % i] = sig.max() - np.abs(sig.min())
        X.loc[seg_id, 'count_big_%d' % i] = len(sig[np.abs(sig) > 500])
        X.loc[seg_id, 'sum_%d' % i] = sig.sum()

        X.loc[seg_id, 'mean_change_rate_first_50000_%d' % i] = np.mean(np.nonzero((np.diff(sig[:50000]) / sig[:50000][:-1]))[0])
        X.loc[seg_id, 'mean_change_rate_last_50000_%d' % i] = np.mean(np.nonzero((np.diff(sig[-50000:]) / sig[-50000:][:-1]))[0])
        X.loc[seg_id, 'mean_change_rate_first_10000_%d' % i] = np.mean(np.nonzero((np.diff(sig[:10000]) / sig[:10000][:-1]))[0])
        X.loc[seg_id, 'mean_change_rate_last_10000_%d' % i] = np.mean(np.nonzero((np.diff(sig[-10000:]) / sig[-10000:][:-1]))[0])

        X.loc[seg_id, 'q95_%d' % i] = np.quantile(sig, 0.95)
        X.loc[seg_id, 'q99_%d' % i] = np.quantile(sig, 0.99)
        X.loc[seg_id, 'q05_%d' % i] = np.quantile(sig, 0.05)
        X.loc[seg_id, 'q01_%d' % i] = np.quantile(sig, 0.01)

        X.loc[seg_id, 'abs_q95_%d' % i] = np.quantile(np.abs(sig), 0.95)
        X.loc[seg_id, 'abs_q99_%d' % i] = np.quantile(np.abs(sig), 0.99)
        X.loc[seg_id, 'abs_q05_%d' % i] = np.quantile(np.abs(sig), 0.05)
        X.loc[seg_id, 'abs_q01_%d' % i] = np.quantile(np.abs(sig), 0.01)

        X.loc[seg_id, 'trend_%d' % i] = add_trend_feature(sig)
        X.loc[seg_id, 'abs_trend_%d' % i] = add_trend_feature(sig, abs_values=True)
        X.loc[seg_id, 'abs_mean_%d' % i] = np.abs(sig).mean()
        X.loc[seg_id, 'abs_std_%d' % i] = np.abs(sig).std()

        X.loc[seg_id, 'mad_%d' % i] = sig.mad()
        X.loc[seg_id, 'kurt_%d' % i] = sig.kurtosis()
        X.loc[seg_id, 'skew_%d' % i] = sig.skew()
        X.loc[seg_id, 'med_%d' % i] = sig.median()

        X.loc[seg_id, 'Hilbert_mean_%d' % i] = np.abs(hilbert(sig)).mean()
        X.loc[seg_id, 'Hann_window_mean'] = (convolve(xc, hann(150), mode='same') / sum(hann(150))).mean()

        X.loc[seg_id, 'classic_sta_lta1_mean_%d' % i] = classic_sta_lta(sig, 500, 10000).mean()
        X.loc[seg_id, 'classic_sta_lta2_mean_%d' % i] = classic_sta_lta(sig, 5000, 100000).mean()
        X.loc[seg_id, 'classic_sta_lta3_mean_%d' % i] = classic_sta_lta(sig, 3333, 6666).mean()
        X.loc[seg_id, 'classic_sta_lta4_mean_%d' % i] = classic_sta_lta(sig, 10000, 25000).mean()

        X.loc[seg_id, 'Moving_average_700_mean_%d' % i] = sig.rolling(window=700).mean().mean(skipna=True)
        X.loc[seg_id, 'Moving_average_1500_mean_%d' % i] = sig.rolling(window=1500).mean().mean(skipna=True)
        X.loc[seg_id, 'Moving_average_3000_mean_%d' % i] = sig.rolling(window=3000).mean().mean(skipna=True)
        X.loc[seg_id, 'Moving_average_6000_mean_%d' % i] = sig.rolling(window=6000).mean().mean(skipna=True)

        ewma = pd.Series.ewm
        X.loc[seg_id, 'exp_Moving_average_300_mean_%d' % i] = ewma(sig, span=300).mean().mean(skipna=True)
        X.loc[seg_id, 'exp_Moving_average_3000_mean_%d' % i] = ewma(sig, span=3000).mean().mean(skipna=True)
        X.loc[seg_id, 'exp_Moving_average_30000_mean_%d' % i] = ewma(sig, span=6000).mean().mean(skipna=True)

        no_of_std = 2
        X.loc[seg_id, 'MA_700MA_std_mean_%d' % i] = sig.rolling(window=700).std().mean()
        X.loc[seg_id, 'MA_700MA_BB_high_mean_%d' % i] = (
                    X.loc[seg_id, 'Moving_average_700_mean_%d' % i] + no_of_std * X.loc[seg_id, 'MA_700MA_std_mean_%d' % i]).mean()
        X.loc[seg_id, 'MA_700MA_BB_low_mean_%d' % i] = (
                    X.loc[seg_id, 'Moving_average_700_mean_%d' % i] - no_of_std * X.loc[seg_id, 'MA_700MA_std_mean_%d' % i]).mean()
        X.loc[seg_id, 'MA_400MA_std_mean_%d' % i] = sig.rolling(window=400).std().mean()
        X.loc[seg_id, 'MA_400MA_BB_high_mean_%d' % i] = (
                    X.loc[seg_id, 'Moving_average_700_mean_%d' % i] + no_of_std * X.loc[seg_id, 'MA_400MA_std_mean_%d' % i]).mean()
        X.loc[seg_id, 'MA_400MA_BB_low_mean_%d' % i] = (
                    X.loc[seg_id, 'Moving_average_700_mean_%d' % i] - no_of_std * X.loc[seg_id, 'MA_400MA_std_mean_%d' % i]).mean()
        X.loc[seg_id, 'MA_1000MA_std_mean_%d' % i] = sig.rolling(window=1000).std().mean()

        X.loc[seg_id, 'iqr_%d' % i] = np.subtract(*np.percentile(sig, [75, 25]))
        X.loc[seg_id, 'q999_%d' % i] = np.quantile(sig, 0.999)
        X.loc[seg_id, 'q001_%d' % i] = np.quantile(sig, 0.001)
        X.loc[seg_id, 'ave10_%d' % i] = stats.trim_mean(sig, 0.1)

    for windows in [10, 100, 1000]:
        x_roll_std = xc.rolling(windows).std().dropna().values
        x_roll_mean = xc.rolling(windows).mean().dropna().values

        X.loc[seg_id, 'ave_roll_std_' + str(windows)] = x_roll_std.mean()
        X.loc[seg_id, 'std_roll_std_' + str(windows)] = x_roll_std.std()
        X.loc[seg_id, 'max_roll_std_' + str(windows)] = x_roll_std.max()
        X.loc[seg_id, 'min_roll_std_' + str(windows)] = x_roll_std.min()
        X.loc[seg_id, 'q01_roll_std_' + str(windows)] = np.quantile(x_roll_std, 0.01)
        X.loc[seg_id, 'q05_roll_std_' + str(windows)] = np.quantile(x_roll_std, 0.05)
        X.loc[seg_id, 'q95_roll_std_' + str(windows)] = np.quantile(x_roll_std, 0.95)
        X.loc[seg_id, 'q99_roll_std_' + str(windows)] = np.quantile(x_roll_std, 0.99)
        X.loc[seg_id, 'av_change_abs_roll_std_' + str(windows)] = np.mean(np.diff(x_roll_std))
        X.loc[seg_id, 'av_change_rate_roll_std_' + str(windows)] = np.mean(
            np.nonzero((np.diff(x_roll_std) / x_roll_std[:-1]))[0])
        X.loc[seg_id, 'abs_max_roll_std_' + str(windows)] = np.abs(x_roll_std).max()

        X.loc[seg_id, 'ave_roll_mean_' + str(windows)] = x_roll_mean.mean()
        X.loc[seg_id, 'std_roll_mean_' + str(windows)] = x_roll_mean.std()
        X.loc[seg_id, 'max_roll_mean_' + str(windows)] = x_roll_mean.max()
        X.loc[seg_id, 'min_roll_mean_' + str(windows)] = x_roll_mean.min()
        X.loc[seg_id, 'q01_roll_mean_' + str(windows)] = np.quantile(x_roll_mean, 0.01)
        X.loc[seg_id, 'q05_roll_mean_' + str(windows)] = np.quantile(x_roll_mean, 0.05)
        X.loc[seg_id, 'q95_roll_mean_' + str(windows)] = np.quantile(x_roll_mean, 0.95)
        X.loc[seg_id, 'q99_roll_mean_' + str(windows)] = np.quantile(x_roll_mean, 0.99)
        X.loc[seg_id, 'av_change_abs_roll_mean_' + str(windows)] = np.mean(np.diff(x_roll_mean))
        X.loc[seg_id, 'av_change_rate_roll_mean_' + str(windows)] = np.mean(
            np.nonzero((np.diff(x_roll_mean) / x_roll_mean[:-1]))[0])
        X.loc[seg_id, 'abs_max_roll_mean_' + str(windows)] = np.abs(x_roll_mean).max()

    return X

Manager function to build the feature fields that are extracted from the acoustic signal, for the training set only. The parameter "proc_id" is the multiprocessing identifier passed in by the multiprocessing caller. This allows for selection of the section of the overall data on which to work. Takes overnight to run 6 processes on the input data. If the "create_features_pk_det" function is called to obtain wavelet generated peak detection features, it may take three days to run. 

In [6]:
def build_fields(proc_id):
    success = 1
    count = 0
    try:
        seg_st = int(NUM_SEG_PER_PROC * proc_id)
        train_df = feather.read_dataframe(os.path.join(DATA_DIR, f'raw_data_{proc_id}.feather'))
        len_df = len(train_df.index)
        start_indices = (np.loadtxt(fname=os.path.join(OUTPUT_DIR, 'start_indices_4k.csv'),
                                    dtype=np.int32, delimiter=','))[:, proc_id]
        train_X = pd.DataFrame(dtype=np.float64)
        train_y = pd.DataFrame(dtype=np.float64, columns=['time_to_failure'])

        len_loop = len(range(seg_st, seg_st + NUM_SEG_PER_PROC))
        for seg_id, start_idx in zip(range(seg_st, seg_st + NUM_SEG_PER_PROC), start_indices):
            count += 1
            print(f'proc_id={proc_id}, count={count}/{len_loop}')
            end_idx = np.int32(start_idx + 150000)
            seg = train_df.iloc[start_idx:end_idx]
            #train_X = create_features_pk_det(seg_id, seg, train_X, start_idx, end_idx)
            train_X = create_features(seg_id, seg, train_X, start_idx, end_idx)
            train_y.loc[seg_id, 'time_to_failure'] = seg['time_to_failure'].values[-1]

        feather.write_dataframe(train_X, os.path.join(OUTPUT_DIR, f'train_x_{proc_id}.feather'))
        feather.write_dataframe(train_y, os.path.join(OUTPUT_DIR, f'train_y_{proc_id}.feather'))

    except:
        print(traceback.format_exc())
        success = 0

    return success  # 1 on success, 0 if fail

Manager function to call the create features functions in multiple processes.  

In [7]:
def run_mp_build():
    num_proc = NUM_THREADS
    pool = mp.Pool(processes=num_proc)
    results = [pool.apply_async(build_fields, args=(pid, )) for pid in range(NUM_THREADS)]
    output = [p.get() for p in results]
    num_built = sum(output)
    pool.close()
    pool.join()
    print(num_built)

In [8]:
run_mp_build()

proc_id=2, count=1/6000
proc_id=5, count=1/6000
proc_id=1, count=1/6000
proc_id=0, count=1/6000
proc_id=4, count=1/6000
proc_id=3, count=1/6000
proc_id=3, count=2/6000
proc_id=5, count=2/6000
proc_id=0, count=2/6000
proc_id=4, count=2/6000
proc_id=2, count=2/6000
proc_id=1, count=2/6000
proc_id=3, count=3/6000
proc_id=5, count=3/6000
proc_id=0, count=3/6000
proc_id=4, count=3/6000
proc_id=1, count=3/6000
proc_id=2, count=3/6000
proc_id=3, count=4/6000
proc_id=0, count=4/6000
proc_id=5, count=4/6000
proc_id=4, count=4/6000
proc_id=1, count=4/6000
proc_id=2, count=4/6000
proc_id=4, count=5/6000
proc_id=3, count=5/6000
proc_id=1, count=5/6000
proc_id=5, count=5/6000
proc_id=2, count=5/6000
proc_id=0, count=5/6000
proc_id=4, count=6/6000
proc_id=1, count=6/6000
proc_id=5, count=6/6000
proc_id=2, count=6/6000
proc_id=3, count=6/6000
proc_id=0, count=6/6000
proc_id=4, count=7/6000
proc_id=1, count=7/6000
proc_id=5, count=7/6000
proc_id=2, count=7/6000
proc_id=3, count=7/6000
proc_id=0, count

proc_id=5, count=55/6000
proc_id=0, count=55/6000
proc_id=4, count=57/6000
proc_id=2, count=57/6000
proc_id=1, count=56/6000
proc_id=4, count=58/6000
proc_id=0, count=56/6000
proc_id=3, count=56/6000
proc_id=5, count=56/6000
proc_id=2, count=58/6000
proc_id=1, count=57/6000
proc_id=4, count=59/6000
proc_id=5, count=57/6000
proc_id=0, count=57/6000
proc_id=3, count=57/6000
proc_id=2, count=59/6000
proc_id=1, count=58/6000
proc_id=4, count=60/6000
proc_id=5, count=58/6000
proc_id=0, count=58/6000
proc_id=3, count=58/6000
proc_id=2, count=60/6000
proc_id=4, count=61/6000
proc_id=5, count=59/6000
proc_id=1, count=59/6000
proc_id=3, count=59/6000
proc_id=0, count=59/6000
proc_id=2, count=61/6000
proc_id=5, count=60/6000
proc_id=4, count=62/6000
proc_id=1, count=60/6000
proc_id=3, count=60/6000
proc_id=0, count=60/6000
proc_id=2, count=62/6000
proc_id=4, count=63/6000
proc_id=5, count=61/6000
proc_id=1, count=61/6000
proc_id=3, count=61/6000
proc_id=0, count=61/6000
proc_id=2, count=63/6000


proc_id=3, count=109/6000
proc_id=1, count=110/6000
proc_id=5, count=111/6000
proc_id=4, count=112/6000
proc_id=2, count=111/6000
proc_id=0, count=109/6000
proc_id=3, count=110/6000
proc_id=1, count=111/6000
proc_id=5, count=112/6000
proc_id=4, count=113/6000
proc_id=2, count=112/6000
proc_id=3, count=111/6000
proc_id=0, count=110/6000
proc_id=1, count=112/6000
proc_id=5, count=113/6000
proc_id=4, count=114/6000
proc_id=2, count=113/6000
proc_id=0, count=111/6000
proc_id=3, count=112/6000
proc_id=1, count=113/6000
proc_id=5, count=114/6000
proc_id=4, count=115/6000
proc_id=0, count=112/6000
proc_id=2, count=114/6000
proc_id=3, count=113/6000
proc_id=1, count=114/6000
proc_id=4, count=116/6000
proc_id=2, count=115/6000
proc_id=5, count=115/6000
proc_id=0, count=113/6000
proc_id=3, count=114/6000
proc_id=1, count=115/6000
proc_id=4, count=117/6000
proc_id=2, count=116/6000
proc_id=0, count=114/6000
proc_id=5, count=116/6000
proc_id=3, count=115/6000
proc_id=4, count=118/6000
proc_id=1, c

proc_id=0, count=162/6000
proc_id=3, count=162/6000
proc_id=2, count=163/6000
proc_id=5, count=164/6000
proc_id=1, count=162/6000
proc_id=4, count=165/6000
proc_id=0, count=163/6000
proc_id=3, count=163/6000
proc_id=2, count=164/6000
proc_id=1, count=163/6000
proc_id=5, count=165/6000
proc_id=4, count=166/6000
proc_id=0, count=164/6000
proc_id=3, count=164/6000
proc_id=2, count=165/6000
proc_id=1, count=164/6000
proc_id=4, count=167/6000
proc_id=5, count=166/6000
proc_id=0, count=165/6000
proc_id=3, count=165/6000
proc_id=2, count=166/6000
proc_id=1, count=165/6000
proc_id=4, count=168/6000
proc_id=5, count=167/6000
proc_id=0, count=166/6000
proc_id=3, count=166/6000
proc_id=2, count=167/6000
proc_id=1, count=166/6000
proc_id=5, count=168/6000
proc_id=4, count=169/6000
proc_id=0, count=167/6000
proc_id=3, count=167/6000
proc_id=1, count=167/6000
proc_id=2, count=168/6000
proc_id=4, count=170/6000
proc_id=5, count=169/6000
proc_id=0, count=168/6000
proc_id=3, count=168/6000
proc_id=1, c

proc_id=0, count=214/6000
proc_id=2, count=215/6000
proc_id=4, count=218/6000
proc_id=3, count=215/6000
proc_id=5, count=217/6000
proc_id=0, count=215/6000
proc_id=1, count=215/6000
proc_id=2, count=216/6000
proc_id=4, count=219/6000
proc_id=5, count=218/6000
proc_id=3, count=216/6000
proc_id=0, count=216/6000
proc_id=1, count=216/6000
proc_id=2, count=217/6000
proc_id=4, count=220/6000
proc_id=3, count=217/6000
proc_id=5, count=219/6000
proc_id=0, count=217/6000
proc_id=1, count=217/6000
proc_id=2, count=218/6000
proc_id=4, count=221/6000
proc_id=5, count=220/6000
proc_id=0, count=218/6000
proc_id=1, count=218/6000
proc_id=3, count=218/6000
proc_id=2, count=219/6000
proc_id=4, count=222/6000
proc_id=5, count=221/6000
proc_id=0, count=219/6000
proc_id=3, count=219/6000
proc_id=1, count=219/6000
proc_id=2, count=220/6000
proc_id=5, count=222/6000
proc_id=4, count=223/6000
proc_id=3, count=220/6000
proc_id=0, count=220/6000
proc_id=1, count=220/6000
proc_id=2, count=221/6000
proc_id=5, c

...

proc_id=0, count=5790/6000
proc_id=2, count=5781/6000
proc_id=5, count=5771/6000
proc_id=1, count=5765/6000
proc_id=3, count=5739/6000
proc_id=0, count=5791/6000
proc_id=4, count=5762/6000
proc_id=2, count=5782/6000
proc_id=5, count=5772/6000
proc_id=3, count=5740/6000
proc_id=0, count=5792/6000
proc_id=1, count=5766/6000
proc_id=4, count=5763/6000
proc_id=2, count=5783/6000
proc_id=3, count=5741/6000
proc_id=5, count=5773/6000
proc_id=0, count=5793/6000
proc_id=1, count=5767/6000
proc_id=4, count=5764/6000
proc_id=2, count=5784/6000
proc_id=0, count=5794/6000
proc_id=5, count=5774/6000
proc_id=1, count=5768/6000
proc_id=3, count=5742/6000
proc_id=4, count=5765/6000
proc_id=2, count=5785/6000
proc_id=0, count=5795/6000
proc_id=5, count=5775/6000
proc_id=4, count=5766/6000
proc_id=3, count=5743/6000
proc_id=1, count=5769/6000
proc_id=2, count=5786/6000
proc_id=4, count=5767/6000
proc_id=0, count=5796/6000
proc_id=3, count=5744/6000
proc_id=1, count=5770/6000
proc_id=5, count=5776/6000
p

proc_id=2, count=5832/6000
proc_id=3, count=5788/6000
proc_id=4, count=5812/6000
proc_id=5, count=5822/6000
proc_id=1, count=5815/6000
proc_id=3, count=5789/6000
proc_id=0, count=5843/6000
proc_id=4, count=5813/6000
proc_id=2, count=5833/6000
proc_id=5, count=5823/6000
proc_id=1, count=5816/6000
proc_id=4, count=5814/6000
proc_id=3, count=5790/6000
proc_id=0, count=5844/6000
proc_id=2, count=5834/6000
proc_id=5, count=5824/6000
proc_id=1, count=5817/6000
proc_id=0, count=5845/6000
proc_id=4, count=5815/6000
proc_id=2, count=5835/6000
proc_id=3, count=5791/6000
proc_id=5, count=5825/6000
proc_id=1, count=5818/6000
proc_id=0, count=5846/6000
proc_id=4, count=5816/6000
proc_id=2, count=5836/6000
proc_id=3, count=5792/6000
proc_id=5, count=5826/6000
proc_id=1, count=5819/6000
proc_id=0, count=5847/6000
proc_id=2, count=5837/6000
proc_id=4, count=5817/6000
proc_id=3, count=5793/6000
proc_id=5, count=5827/6000
proc_id=1, count=5820/6000
proc_id=4, count=5818/6000
proc_id=2, count=5838/6000
p

proc_id=3, count=5837/6000
proc_id=5, count=5873/6000
proc_id=0, count=5893/6000
proc_id=2, count=5885/6000
proc_id=1, count=5865/6000
proc_id=4, count=5863/6000
proc_id=3, count=5838/6000
proc_id=5, count=5874/6000
proc_id=1, count=5866/6000
proc_id=0, count=5894/6000
proc_id=2, count=5886/6000
proc_id=4, count=5864/6000
proc_id=3, count=5839/6000
proc_id=5, count=5875/6000
proc_id=0, count=5895/6000
proc_id=1, count=5867/6000
proc_id=2, count=5887/6000
proc_id=4, count=5865/6000
proc_id=3, count=5840/6000
proc_id=5, count=5876/6000
proc_id=0, count=5896/6000
proc_id=2, count=5888/6000
proc_id=1, count=5868/6000
proc_id=4, count=5866/6000
proc_id=3, count=5841/6000
proc_id=5, count=5877/6000
proc_id=2, count=5889/6000
proc_id=0, count=5897/6000
proc_id=1, count=5869/6000
proc_id=4, count=5867/6000
proc_id=3, count=5842/6000
proc_id=5, count=5878/6000
proc_id=2, count=5890/6000
proc_id=1, count=5870/6000
proc_id=0, count=5898/6000
proc_id=4, count=5868/6000
proc_id=3, count=5843/6000
p

proc_id=2, count=5936/6000
proc_id=5, count=5924/6000
proc_id=0, count=5943/6000
proc_id=1, count=5915/6000
proc_id=4, count=5913/6000
proc_id=3, count=5889/6000
proc_id=2, count=5937/6000
proc_id=0, count=5944/6000
proc_id=5, count=5925/6000
proc_id=1, count=5916/6000
proc_id=4, count=5914/6000
proc_id=3, count=5890/6000
proc_id=0, count=5945/6000
proc_id=2, count=5938/6000
proc_id=1, count=5917/6000
proc_id=5, count=5926/6000
proc_id=4, count=5915/6000
proc_id=3, count=5891/6000
proc_id=0, count=5946/6000
proc_id=2, count=5939/6000
proc_id=1, count=5918/6000
proc_id=5, count=5927/6000
proc_id=4, count=5916/6000
proc_id=2, count=5940/6000
proc_id=3, count=5892/6000
proc_id=0, count=5947/6000
proc_id=5, count=5928/6000
proc_id=1, count=5919/6000
proc_id=4, count=5917/6000
proc_id=2, count=5941/6000
proc_id=3, count=5893/6000
proc_id=0, count=5948/6000
proc_id=5, count=5929/6000
proc_id=1, count=5920/6000
proc_id=4, count=5918/6000
proc_id=3, count=5894/6000
proc_id=2, count=5942/6000
p

proc_id=1, count=5966/6000
proc_id=5, count=5974/6000
proc_id=4, count=5962/6000
proc_id=0, count=5996/6000
proc_id=2, count=5987/6000
proc_id=1, count=5967/6000
proc_id=3, count=5939/6000
proc_id=5, count=5975/6000
proc_id=4, count=5963/6000
proc_id=2, count=5988/6000
proc_id=0, count=5997/6000
proc_id=3, count=5940/6000
proc_id=5, count=5976/6000
proc_id=1, count=5968/6000
proc_id=4, count=5964/6000
proc_id=0, count=5998/6000
proc_id=2, count=5989/6000
proc_id=3, count=5941/6000
proc_id=5, count=5977/6000
proc_id=1, count=5969/6000
proc_id=4, count=5965/6000
proc_id=0, count=5999/6000
proc_id=2, count=5990/6000
proc_id=1, count=5970/6000
proc_id=3, count=5942/6000
proc_id=5, count=5978/6000
proc_id=4, count=5966/6000
proc_id=0, count=6000/6000
proc_id=1, count=5971/6000
proc_id=2, count=5991/6000
proc_id=3, count=5943/6000
proc_id=5, count=5979/6000
proc_id=4, count=5967/6000
proc_id=2, count=5992/6000
proc_id=1, count=5972/6000
proc_id=5, count=5980/6000
proc_id=4, count=5968/6000
p

In [21]:
def join_mp_build():
    
    df0 = feather.read_dataframe(os.path.join(OUTPUT_DIR, 'train_x_%d.feather' % 0))
    df1 = feather.read_dataframe(os.path.join(OUTPUT_DIR, 'train_y_%d.feather' % 0))

    for i in tqdm_notebook(range(1, NUM_THREADS)):
        print('working %d' % i)
        temp = feather.read_dataframe(os.path.join(OUTPUT_DIR, 'train_x_%d.feather' % i))
        df0 = df0.append(temp)
        temp = feather.read_dataframe(os.path.join(OUTPUT_DIR, 'train_y_%d.feather' % i))
        df1 = df1.append(temp)

    feather.write_dataframe(df0, os.path.join(OUTPUT_DIR, 'train_x.feather'))
    feather.write_dataframe(df1, os.path.join(OUTPUT_DIR, 'train_y.feather'))

In [22]:
join_mp_build()

working 1
working 2
working 3
working 4
working 5


Build features from the Kaggle test data files. This produces the test file that will be used for prediction and submission to Kaggle. If the "create_features_pk_det" function is called to obtain wavelet generated peak detection features, it may take two days to run. 

In [27]:
def build_test_fields():
    train_X = feather.read_dataframe(os.path.join(OUTPUT_DIR, 'train_x.feather'))
    
    try:
        train_X.drop(labels=['seg_id', 'seg_start', 'seg_end'], axis=1, inplace=True)
    except:
        pass

    submission = pd.read_csv(os.path.join(DATA_DIR, 'sample_submission.csv'), index_col='seg_id')
    test_X = pd.DataFrame(columns=train_X.columns, dtype=np.float64, index=submission.index)

    print('start for loop')
    count = 0
    for seg_id in tqdm_notebook(test_X.index):
        seg = pd.read_csv(os.path.join(DATA_DIR, 'test', str(seg_id) + '.csv'))
        #train_X = create_features_pk_det(seg_id, seg, train_X, start_idx, end_idx)
        test_X = create_features(seg_id, seg, test_X, 0, 0)
        
    feather.write_dataframe(test_X, os.path.join(OUTPUT_DIR, 'test_x.feather'))

In [28]:
build_test_fields()

start for loop


Scale the features. This appeared to help, even with gradient boosted decision tree algorithms and is necessary with many other machine learning algorithms. 

In [29]:
def scale_fields(fn_train='train_x.feather',
                 fn_test='test_x.feather', 
                 fn_out_train='scaled_train_X.feather',
                 fn_out_test='scaled_test_X.feather'):

    train_X = feather.read_dataframe(os.path.join(OUTPUT_DIR, fn_train))
    try:
        train_X.drop(labels=['seg_id', 'seg_start', 'seg_end'], axis=1, inplace=True)
    except:
        pass
    test_X = feather.read_dataframe(os.path.join(OUTPUT_DIR, fn_test))

    print('start scaler')
    scaler = StandardScaler()
    scaler.fit(train_X)
    scaled_train_X = pd.DataFrame(scaler.transform(train_X), columns=train_X.columns)
    scaled_test_X = pd.DataFrame(scaler.transform(test_X), columns=test_X.columns)

    feather.write_dataframe(scaled_train_X, os.path.join(OUTPUT_DIR, fn_out_train))
    feather.write_dataframe(scaled_test_X, os.path.join(OUTPUT_DIR, fn_out_test))

Put the feature creation functions together and create the features. Some of these functions can take a long time to run, so it is recommended that it be done from an IDE and one function at a time. If it fails part way down due to a path name being wrong then it is not necessary to re-run every function. 

In [30]:
scale_fields()

start scaler


### Building the model

In [48]:
maes = []
rmses = []
tr_maes = []
tr_rmses = []
submission = pd.read_csv(os.path.join(DATA_DIR, 'sample_submission.csv'), index_col='seg_id')

scaled_train_X = feather.read_dataframe(os.path.join(OUTPUT_DIR, 'scaled_train_X.feather'))
#df = pd.read_csv(r'pk8/scaled_train_X_8_slope.csv')  # adds in wavelet features
#scaled_train_X = scaled_train_X.join(df)

scaled_test_X = feather.read_dataframe(os.path.join(OUTPUT_DIR, 'scaled_test_X.feather'))
#df = pd.read_csv(r'pk8/scaled_test_X_8_slope.csv')  # adds in wavelet features
#scaled_test_X = scaled_test_X.join(df)

In [73]:
pcol = []
pcor = []
pval = []
y = feather.read_dataframe(os.path.join(OUTPUT_DIR, 'train_y.feather'))

# use pearson's to eliminate features with suspect correlation - helped kaggle scores
for col in scaled_train_X.columns:
    pcol.append(col)
    pcor.append(abs(pearsonr(scaled_train_X[col], y.squeeze())[0]))
    pval.append(abs(pearsonr(scaled_train_X[col], y.squeeze())[1]))

df = pd.DataFrame(data={'col': pcol, 'cor': pcor, 'pval': pval}, index=range(len(pcol)))
df.sort_values(by=['cor', 'pval'], inplace=True)
df.dropna(inplace=True)
df = df.loc[df['pval'] <= 0.05]
                                                                                                        


drop_cols = []

for col in scaled_train_X.columns:
    if col not in df['col'].tolist():
        drop_cols.append(col)

scaled_train_X.drop(labels=drop_cols, axis=1, inplace=True)
scaled_test_X.drop(labels=drop_cols, axis=1, inplace=True)

train_y = feather.read_dataframe(os.path.join(OUTPUT_DIR, 'train_y.feather'))

predictions = np.zeros(len(scaled_test_X))
preds_train = np.zeros(len(scaled_train_X))

print('shapes of train and test:', scaled_train_X.shape, scaled_test_X.shape)

# as an alternate, select 6 fold and shuffle=False for non-overlap cross validation
n_fold = 8
folds = KFold(n_splits=n_fold, shuffle=True, random_state=42)

for fold_, (trn_idx, val_idx) in tqdm_notebook(enumerate(folds.split(scaled_train_X, train_y.values))):
    print('working fold %d' % fold_)
    t0 = time.time()
    strLog = "fold {}".format(fold_)
    print(strLog)

    X_tr, X_val = scaled_train_X.iloc[trn_idx], scaled_train_X.iloc[val_idx]
    y_tr, y_val = train_y.iloc[trn_idx], train_y.iloc[val_idx]

    # use commenting to select lgb or xgb model
    # model = lgb.LGBMRegressor(**params, n_estimators=60000, n_jobs=-1)
    # model.fit(X_tr, y_tr,
    #           eval_set=[(X_tr, y_tr), (X_val, y_val)], eval_metric='mae',
    #           verbose=1000, early_stopping_rounds=200)

    model = xgb.XGBRegressor(n_estimators=1000,
                                   learning_rate=0.1,
                                   max_depth=6,
                                   subsample=0.9,
                                   colsample_bytree=0.67,
                                   reg_lambda=1.0, # seems best within 0.5 of 2.0
                                   # gamma=1,
                                   random_state=777+fold_,
                                   n_jobs=12,
                                   verbosity=2)
    model.fit(X_tr, y_tr)

    # predictions
    preds = model.predict(scaled_test_X)  #, num_iteration=model.best_iteration_)  # uncomment for lgb, and below
    predictions += preds / folds.n_splits
    preds = model.predict(scaled_train_X)  #, num_iteration=model.best_iteration_)
    preds_train += preds / folds.n_splits

    # validation error
    # mean absolute error
    preds = model.predict(X_val)  # , num_iteration=model.best_iteration_)
    mae = mean_absolute_error(y_val, preds)
    print('MAE: %.6f' % mae)
    maes.append(mae)

    # root mean squared error
    rmse = mean_squared_error(y_val, preds)
    print('RMSE: %.6f' % rmse)
    rmses.append(rmse)

    # training error, monitor for over fit - still an area for improvement despite great kaggle scores
    preds = model.predict(X_tr)  #, num_iteration=model.best_iteration_)

    mae = mean_absolute_error(y_tr, preds)
    print('Tr MAE: %.6f' % mae)
    tr_maes.append(mae)

    rmse = mean_squared_error(y_tr, preds)
    print('Tr RMSE: %.6f' % rmse)
    tr_rmses.append(rmse)
    print('loop time:', time.time() - t0)

# print out statistics for validation and training sets (training monitors overfitting)
print('MAEs', maes)
print('MAE mean: %.6f' % np.mean(maes))
print('RMSEs', rmses)
print('RMSE mean: %.6f' % np.mean(rmses))

print('Tr MAEs', tr_maes)
print('Tr MAE mean: %.6f' % np.mean(tr_maes))
print('Tr RMSEs', rmses)
print('Tr RMSE mean: %.6f' % np.mean(tr_rmses))

# reset file names as needed
submission.time_to_failure = predictions
submission.to_csv('submission_xgb_slope_pearson_6fold.csv')  # index needed, it is seg id

pr_tr = pd.DataFrame(data=preds_train, columns=['time_to_failure'], index=range(0, preds_train.shape[0]))
pr_tr.to_csv(r'preds_tr_xgb_slope_pearson.csv', index=False)
print('Train shape: {}, Test shape: {}, Y shape: {}'.format(scaled_train_X.shape, scaled_test_X.shape, train_y.shape))

shapes of train and test: (36000, 617) (2624, 617)


working fold 0
fold 0
MAE: 0.771981
RMSE: 1.219663
Tr MAE: 0.191268
Tr RMSE: 0.066732
loop time: 82.86960196495056
working fold 1
fold 1
MAE: 0.760499
RMSE: 1.213693
Tr MAE: 0.185753
Tr RMSE: 0.063012
loop time: 83.91227531433105
working fold 2
fold 2
MAE: 0.767898
RMSE: 1.213613
Tr MAE: 0.184281
Tr RMSE: 0.062159
loop time: 84.43904161453247
working fold 3
fold 3
MAE: 0.770245
RMSE: 1.198317
Tr MAE: 0.188560
Tr RMSE: 0.064824
loop time: 83.9866840839386
working fold 4
fold 4
MAE: 0.746642
RMSE: 1.177099
Tr MAE: 0.187302
Tr RMSE: 0.063652
loop time: 84.28514885902405
working fold 5
fold 5
MAE: 0.775217
RMSE: 1.301750
Tr MAE: 0.183662
Tr RMSE: 0.061196
loop time: 84.38548135757446
working fold 6
fold 6
MAE: 0.760649
RMSE: 1.235512
Tr MAE: 0.185509
Tr RMSE: 0.063045
loop time: 83.58263635635376
working fold 7
fold 7
MAE: 0.746306
RMSE: 1.158549
Tr MAE: 0.189663
Tr RMSE: 0.065264
loop time: 83.95857453346252
MAEs [0.7719806553246453, 0.7604988428992737, 0.7678980882260948, 0.7702447135407

In [75]:
print('MAE mean: %.6f' % np.mean(maes))
print('MAE std: %.6f' % np.std(maes))

MAE mean: 0.762429
MAE std: 0.010379
